In [1]:
!pip install langchain langchain-openai  python-dotenv --quiet

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

model_name = "gpt-4.1-mini"


In [16]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(api_version="2024-12-01-preview",model= model_name)

In [6]:
from pydantic import BaseModel
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

parser = PydanticOutputParser(pydantic_object=MeetingNotes)

In [17]:
from langchain_core.prompts import PromptTemplate

def extract_meeting_notes(transcript: str):

    prompt = PromptTemplate.from_template("""
        You are a meeting assistant.
        1. Summarize the meeting transcription below in exactly two sentences.
        2. Then list all action items mentioned, each as a separate bullet beginning with a dash.

        Transcript:
        {transcript}

        Return the result strictly in this JSON format:
        {format_instructions}
    """)

    chain = prompt | llm | parser
    output = chain.invoke({
        "transcript":transcript,
        "format_instructions": parser.get_format_instructions()
    })

    return output.model_dump()

In [18]:
extract_meeting_notes("""Host: Let’s kick off our marketing sync.  
Emma: The social campaign draft is 80% done; I’ll share it today.  
Frank: I spoke with the design team—they’ll deliver assets by Tuesday.  
Emma: Once we have assets, I’ll schedule the ads for next week.  
George: Reminder: submit your budget requests before end of day.  
Host: Noted. I’ll send out the final budget spreadsheet.  
""")

{'summary': 'The marketing team discussed the progress on the social campaign draft and the timeline for design asset delivery. Budget submission deadlines were emphasized, and final budget documents will be circulated soon.',
 'action_items': ['Emma to share the social campaign draft today',
  'Design team to deliver assets by Tuesday',
  'Emma to schedule ads once assets are received',
  'All team members to submit budget requests before end of day',
  'Host to send out the final budget spreadsheet']}

In [ ]:
extract_meeting_notes("""Alice: Welcome everyone. Today we need to finalize the Q3 roadmap.  
Bob: I’ve emailed the updated feature list—please review by Friday.  
Carol: I’ll set up the user‐testing sessions next week.  
Dan: Let’s push the new UI mockups to staging on Wednesday.  
Alice: Great. Also, can someone compile the stakeholder feedback into a slide deck?  
Bob: I can handle the slide deck by Monday.  
Alice: Thanks, team. Meeting adjourned.  
""")

{'summary': 'The team discussed finalizing the Q3 roadmap, including reviewing the updated feature list, scheduling user-testing sessions, and preparing UI mockups. Additionally, tasks such as compiling stakeholder feedback into a slide deck were assigned to ensure progress before the next deadlines.',
 'action_items': ['Review the updated feature list by Friday.',
  'Set up user-testing sessions next week.',
  'Push the new UI mockups to staging on Wednesday.',
  'Compile the stakeholder feedback into a slide deck by Monday.']}

In [20]:
extract_meeting_notes("""Priya: We’ve identified a bottleneck in the checkout API.  
Carlos: I’ll benchmark the current response times by tomorrow.  
Priya: After that, optimize the database indices.  
Carlos: Sure, I’ll propose index changes by Thursday.  
Sara: Also, draft a rollback plan in case of production issues.  
Carlos: I’ll include that in my ticket.  
""")

{'summary': 'The team has identified a bottleneck in the checkout API and plans to benchmark current response times, optimize database indices, and prepare a rollback plan. Carlos will lead the benchmarking, propose index changes, and include the rollback plan in his ticket.',
 'action_items': ['Benchmark the current response times by tomorrow',
  'Optimize the database indices after benchmarking',
  'Propose index changes by Thursday',
  'Draft a rollback plan in case of production issues and include it in the ticket']}

In [21]:
extract_meeting_notes("""HR: We need to update the travel‐expense policy.  
Alex: I’ll draft the new policy doc and share with legal.  
HR: Please include guidelines on per diem limits.  
Legal: I’ll review and provide feedback within three days.  
HR: Once finalized, schedule a companywide announcement.  
""")

{'summary': 'The team discussed updating the travel-expense policy, including adding guidelines on per diem limits. Alex will draft the policy and share it with Legal for review, after which a companywide announcement will be scheduled.',
 'action_items': ['Alex to draft the new travel-expense policy document and share it with Legal.',
  'Include guidelines on per diem limits in the policy draft.',
  'Legal to review the draft and provide feedback within three days.',
  'Schedule a companywide announcement once the policy is finalized.']}

In [24]:
extract_meeting_notes("test")

{'summary': "The transcript provided contained only the word 'test' and did not include any substantive discussion or content. Therefore, no key topics or points were addressed during this meeting.",
 'action_items': []}